In [ ]:
#@title Imports and Installations { vertical-output: true, form-width: "15%" }
from fileinput import filename
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo as pyo
from pytest import param
import pandas as pd
import numpy as np
import os.path
import math
from datetime import datetime
from multiprocessing import Process

In [ ]:
#for google colab
#@title Optimization Model { vertical-output: true, form-width: "15%" }

class greenHydrogenProduction:
    def writeDataFile(dataFileName,inputDataset):
        with open('../modelInputs/'+str(dataFileName)+'.dat', 'w') as f:
            
            
            #horizon (time) set
            f.write('set horizon := ')
            for i in range(len(inputDataset["cfWind"])):
                f.write('%d ' % i)
            f.write(';\n\n')

            #ey plants set
            f.write('set eyPlants := ')
            for i in range(len(inputDataset["capexEY"])):
                f.write('%d ' % i)
            f.write(';\n\n')
            
            
            #simplifying writing .dat file with for loop
            paramNames = inputDataset.keys()
            
            #single param index names-for writing correct structure of .dat file
            singleParamIndexNames = ["cfWind","cfSolar","capexEY","fixedOpexEY","variableOpexEY","energyUseEY","stackSize"]
            
            
            for paramName in paramNames:
                if((paramName in ["horizon","eyPlants"])):
                    #skip names as they are sets defined above
                    continue
                elif(paramName in singleParamIndexNames):
                    #writing correct pyomo structure for re generation
                    f.write('param %s := \n' % (paramName))
                    for i in range(len(inputDataset[paramName])):
                        if(i != len(inputDataset[paramName])-1):
                            f.write('%d %f \n' % (i,inputDataset[paramName][i]))
                        else:
                            f.write('%d %f' % (i,inputDataset[paramName][i]))                    
                    f.write(';\n\n')
                else:
                    #all other parameters are single values
                    f.write('param %s := %f; \n' % (paramName,inputDataset[paramName]))
            
            print("Completed data file")
            

    def main(dataFileName,inputDataset,testMode=False):
        """Hourly operations of green hydrogen production complex

        Args:
            dataFileName (str): .dat file name which will read in/be created data for model run
            inputDataset (dict): see README for further clarification on what parameters are expected to be inputted into spreadsheet
            testMode (bool): automatically set to false, if true-will delete the .dat input file and output file associated with the dataFileName
        """ 
        #deleting files if test mode is activated
        if(testMode):
            try:
                os.remove(f"../modelInputs/{dataFileName}.dat")
                os.remove(f"../modelOutputs/{dataFileName}.xlsx")
            except:
                print("Test mode activated but one of files already deleted")
        
            
        
        # creating optimization model with pyomo abstract representation
        model = AbstractModel()

        ################### START SETS  ###################
        #timesteps in simulation-based on capacity factor dataset
        model.horizon = RangeSet(0,len(inputDataset["cfSolar"])-1)
        
        #number of unique electroyzer (ey) models to select from
        model.eyPlants = RangeSet(0,len(inputDataset["capexEY"])-1)
        
        ################### END SETS  ###################


        
        ################### START PARAMETERS  ###################
        #hydrogen demand on daily basis (differs form formulation which does some calculations before hand)
        model.hydrogenDemand = Param()
        
        #respective capacity factor timeseries data for wind and solar site
        model.cfWind = Param(model.horizon)
        model.cfSolar = Param(model.horizon)
        
        #chemical efficiency of storing hydrogen in hydrogen tank
        model.hsStoreEfficiency = Param()
        
        #chemical efficiency of deploying hydrogen from tank
        model.hsDeployEfficiency = Param()

        #energy efficiency of storing energy in battery (e.g. you need to put in 1.2 kW to store 1 kW)
        model.bsStoreEfficiency = Param()
        
        #energy efficiency of deploying energy from battery
        model.bsDeployEfficiency = Param()        
        
        #vaporization rate of H2 in percent stored per timestep
        model.hsVaporizationRate = Param()
        
                
        #Looking at CAPEX for wind and solar per MW
        model.capexWind = Param()
        model.capexSolar = Param()
        
        #capex for electroysis depending on plant size (to capture economics of scale)
        model.capexEY = Param(model.eyPlants)

        #battery storage power CAPEX per MWh
        model.capexBSpower = Param()
        
        #battery storage energy CAPEX per MW
        model.capexBSenergy = Param()
        
        #same outline above for different stages but now looking at fixed OPEX
        model.fixedOpexWind = Param()
        model.fixedOpexSolar = Param()
        model.fixedOpexEY = Param(model.eyPlants)
        model.fixedOpexBSpower = Param()
        
        #Now only looking at variable OPEX for EY as all the other technologies are assumed to have 
        #negligible variable OPEX
        model.variableOpexEY = Param(model.eyPlants)

       
       
        #energy consumption for each EY model type MWh/kg H2
        model.energyUseEY = Param(model.eyPlants)
        
        
        #maximum operating percentage of nameplate capacity for BS
        model.maxCapacityBS = Param()    
        
        #minimum operating percentage of nameplate capacity for BS
        model.minCapacityBS = Param()    

        #minimum operating capacity of electroyzer
        model.minCapacityEY = Param()                      
       
        #stack size (rated energy for EY-MW) from each EY model type
        model.stackSize = Param(model.eyPlants)
        
        #number of years the simulation will run through
        model.simulationLifetime = Param()

        #lifetime of renewable plants
        model.reLifetime = Param()
        
        #lifetime of electroyzers (all treated the same)
        model.eyLifetime = Param()
        
        #lifetime of battery storage
        model.bsLifetime = Param()
        
        #WACC or discount rate for plant operations
        model.WACC = Param()
        
        #utilization ratio of plant (not used in model but calculated for total hydrogen production)
        model.utilizationRatio = Param()
        ################### END PARAMETERS  ###################
        
        
        
        
        ################### START DECISION VARIABLES  ###################
        #how much wind capacity to build
        model.windCapacity = Var(domain=NonNegativeReals)
        
        #how much solar capacity to build
        model.solarCapacity = Var(domain=NonNegativeReals)
        
        #how many stacks to build of model i for EY (so only integers)
        model.eyCapacity = Var(model.eyPlants, domain = NonNegativeIntegers)
        
        #total battery power storage capacity to build (MWh)
        model.bsPowerCapacity = Var(domain=NonNegativeReals)
        
        #total battery energy storage capacity to build (MW)
        model.bsEnergyCapacity = Var(domain=NonNegativeReals)
        
        #H2 (kg) to produce from models i at timestep t
        model.eyGen = Var(model.eyPlants,model.horizon,domain=NonNegativeReals)

        #amount of hydrogen (kg) to store in tanks at timestep t
        model.hsStore = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of energy (MWh) to store in battery storage at timestep t
        model.bsStore = Var(model.horizon,domain=NonNegativeReals)      
        
        #amount of hydrogen (kg) in storage at timestep t which can be used
        model.hsAvail = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of energy (MWh) available in battery at timestep t
        model.bsAvail = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of hydrogen (kg) to deploy to HB process at timestep t
        model.hsDeploy = Var(model.horizon,domain=NonNegativeReals)
        
        #amount of energy (MWh) to release into islanded grid at timestep t
        model.bsDeploy = Var(model.horizon,domain=NonNegativeReals)
    
        ################### END DECISION VARIABLES    ###################
        
        
        
###################     START OBJECTIVE     ###################
        #sum up the CAPEX, fixed OPEX, and variable OPEX costs for each of the 7 components of the green ammonia value chain
        def windCosts(model):
            #sum of capex multiplied by wind capacity built + 
            #operational fixed costs*number of plant years that the OPEX is applied and then factor in time value of money
            return (model.windCapacity*(sum((model.capexWind / (math.pow((1+model.WACC),j*model.reLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.reLifetime))))
                                        + sum((model.fixedOpexWind/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))))
             
        def solarCosts(model):
            #sum of capex multiplied by solar capacity built + 
            #operational fixed costs*number of plant years that the OPEX is applied (same process as in wind)
            return (model.solarCapacity*(sum((model.capexSolar / (math.pow((1+model.WACC),j*model.reLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.reLifetime))))
                                         + sum((model.fixedOpexSolar/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))))

        def eyCosts(model):
            # have a temporary fix for running model without annual data for variable OPEX(8760/len*+(model.horizon)))-calculates the scaling factor for full year expenditures
            # stackSize[i]*eyCapacity[i] = total MW consumption of ey model i
            # need to then multiply by capexEY (in $/MW) and fixed OPEX ($/MW) including discount factor
            # for variable ($/kg H2) multiply by generation at each hour (this could be the water cost)
            return (sum(model.stackSize[i]*model.eyCapacity[i]*(sum((model.capexEY[i] / (math.pow((1+model.WACC),j*model.eyLifetime)) for j in np.arange(math.ceil(model.simulationLifetime/model.eyLifetime)))) + 
                    sum((model.fixedOpexEY[i]/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime))))  for i in model.eyPlants) +
                    sum((8760/len(model.horizon))*sum(model.variableOpexEY[i]*model.eyGen[i,t] for i in model.eyPlants)/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))
        
        def bsCosts(model):
            #same as hsCosts but dont have to change bsCapacity to MW as already in MWh
            return ((sum((model.bsPowerCapacity*(model.capexBSpower / (math.pow((1+model.WACC),j*model.bsLifetime)))+ model.bsEnergyCapacity*(model.capexBSenergy / (math.pow((1+model.WACC),j*model.bsLifetime)))) for j in np.arange(math.ceil(model.simulationLifetime/model.bsLifetime))) + 
                    model.bsPowerCapacity*sum((model.fixedOpexBSpower/(math.pow((1+model.WACC),t)) for t in np.arange(model.simulationLifetime)))))                      

      
        
        def minCost_rule(model):
            return (windCosts(model) + solarCosts(model) + eyCosts(model) + 
                    bsCosts(model))
        
        model.SystemCost = Objective(rule = minCost_rule, sense = minimize)
        
        ###################       END OBJECTIVE     ###################
    
        ###################       START CONSTRAINTS     ###################
        #meet the hydrogen production targets for demand over the entire production basis (converting total simulation run multiplied by ratio to convert to daily basis)
        def meetHydrogenDemand(model):
            return(sum((sum(model.eyGen[i,t] for i in model.eyPlants))for t in model.horizon) 
                   == math.ceil(len(model.horizon)/24)*model.hydrogenDemand)
        model.meetHydrogenDemandConstraint = Constraint(rule=meetHydrogenDemand)

        #generate enough energy to meet all required islanded components demand
        #Note: the energy usage parameters should encapsulate the energy efficiencies of each of the stages
        def energyDemand(model,t):
            #summing up all the demand from the various energy consumption stages (MWh/kg*kg produced gives us MWh)
            return (sum(model.energyUseEY[i]*model.eyGen[i,t] for i in model.eyPlants) + (model.bsStore[t]/model.bsStoreEfficiency))
        
        def energyGen(model,t):
            #summing up generation from wind, solar, and battery storage (don't need to include efficiency for bs as 
            # the decision variable BSdeploy is the actual quantity deployed to grid)
            return(model.cfWind[t]*model.windCapacity + model.cfSolar[t]*model.solarCapacity + model.bsDeploy[t])
        
        def energyRule(model,t):
            #energy generated should always be equal to or greater than energy demanded
            return(energyDemand(model,t) <= energyGen(model,t))
        
        model.energyConstraint = Constraint(model.horizon,rule=energyRule)


        #battery storage operations definition (current energy available is 
        # 30% of rated energy capacity at beginning and equal to previous available amount + new energy stored - (energy deployed + energy required to deploy it))
        def bsAvailEnergyRule(model,t):
            if t == 0:
                return(model.bsAvail[0] == .3*model.bsEnergyCapacity)
            else:
                return(model.bsAvail[t] == model.bsAvail[t-1] + model.bsStore[t-1] - (model.bsDeploy[t-1])/model.bsDeployEfficiency)
        model.bsAvailEnergyDefConstraint = Constraint(model.horizon,rule=bsAvailEnergyRule)

        #max available energy you can have in storage is storage capacity multiplied by max operating percentage (usually 95%)
        def bsAvailUpperBoundRule(model,t):
            return(model.bsAvail[t] <= model.maxCapacityBS*model.bsEnergyCapacity)
        model.bsAvailUpperBoundConstraint = Constraint(model.horizon,rule=bsAvailUpperBoundRule)

        #max state of charge of battery is storage capacity multiplied by min operating percentage
        def bsAvailLowerBoundRule(model,t):
            return(model.bsAvail[t] >= model.minCapacityBS*model.bsEnergyCapacity)
        model.bsAvailLowerBoundConstraint = Constraint(model.horizon,rule=bsAvailLowerBoundRule)
        
        
        #max amount of energy you can store in bs is max capacity - current energy charge at start - energy you deploy in hour
        #for simplicity, I assume you can not store energy in the hour and deploy it within the same hour-this would require finer resolution then hourly capacity factors
        def bsStoreEnergyUpperBoundRule(model,t):
            return(model.bsStore[t] <= model.bsEnergyCapacity - model.bsAvail[t])
        model.bsStoreEnergyUpperBoundConstraint = Constraint(model.horizon,rule=bsStoreEnergyUpperBoundRule)

        #max amount of power you can store or deploy in bs in an hour
        def bsPowerUpperBoundRule(model,t):
            return(model.bsStore[t] + model.bsDeploy[t]/model.bsDeployEfficiency <= model.bsPowerCapacity)
        model.bsPowerUpperBoundConstraint = Constraint(model.horizon,rule=bsPowerUpperBoundRule)
        
        
        #energy deployed (and required) from storage must be less than energy available
        def bsDeployUpperBoundRule(model,t):
            return(model.bsDeploy[t]/model.bsDeployEfficiency <= model.bsAvail[t])
                
        model.bsDeployUpperBoundConstraint = Constraint(model.horizon,rule=bsDeployUpperBoundRule)

        #maximum energy to power ratio of 10 hours
        def bsPowerEnergyRatioMaxRule(model):
            return(model.bsEnergyCapacity <= 10*model.bsPowerCapacity)
        model.bsPowerEnergyRatioMaxConstraint = Constraint(rule=bsPowerEnergyRatioMaxRule)


        #hydrogen production (in kg) from all the stacks in model i must be less than total number built*output per unit (have to convert stackSize (MW) to kg (and thus divide by energyUsage (MWh/kg)))
        #includes running at 110% of capacity        
        def eyGenUpperBoundRule(model,i,t):
            return(model.eyGen[i,t] <= 1.1*(model.stackSize[i]/model.energyUseEY[i])*model.eyCapacity[i])
        model.eyGenUpperBoundConstraint = Constraint(model.eyPlants,model.horizon,rule=eyGenUpperBoundRule)
        
        ###################       END   CONSTRAINTS     ###################
    


        ###################          WRITING DATA       ###################
        if(os.path.isfile(f"../dataInputs/{dataFileName}.dat")):
            print(f"Data file {dataFileName} already exists!\nSkipping creating .dat file")
        else:
            #print(f"Data file {dataFileName} does not exist.\nCreating .dat file")
            greenHydrogenProduction.writeDataFile(dataFileName,inputDataset)
        
        # load in data for the system
        data = DataPortal()
        data.load(filename=f"../modelInputs/{dataFileName}.dat", model=model)
        instance = model.create_instance(data)
                
        
        
        solver = SolverFactory('glpk')
        result = solver.solve(instance)
        #instance.display()
        
        
        #setting up structure in order to get out decision variables (and objective) and save in correct excel format
        singleDecisionVariables = ["windCapacity","solarCapacity","bsPowerCapacity","bsEnergyCapacity",
                                        "totalSystemCost","LCOH","windCosts",
                                        "solarCosts","eyCosts","bsPowerCosts","bsEnergyCosts",
                                        "windCapexCosts","windOpexCosts","solarCapexCosts",
                                        "solarOpexCosts","eyCapexCosts","eyOpexCosts",
                                        "bsPowerCapexCosts", "bsPowerOpexCosts"]
        
        
        
        #included wind and solar generation for simplifying data analysis and timestep
        hourlyDecisionVariables = ["windGen","solarGen","bsStore",
                                   "bsAvail","bsDeploy","timestep"]
        
        #eyDecisionVariables =  ["eyCapacity"]
        
        #eyHourlyDecisionVariables = ["eyGen"]
        

        singleDvDataset = pd.DataFrame(0.0, index=np.arange(1), columns=singleDecisionVariables)

        hourlyDvDataset = pd.DataFrame(0.0, index=np.arange(len(inputDataset["cfSolar"])), columns=hourlyDecisionVariables)
        
        eySingleDvDataset = pd.DataFrame(0.0, index=np.arange(len(inputDataset["capexEY"])), columns=np.arange(len(inputDataset["capexEY"]))) 
        
        eyHourlyDvDataset = pd.DataFrame(0.0, index=np.arange(len(inputDataset["cfSolar"])), columns=np.arange(len(inputDataset["capexEY"]))) 
        
        #assigning single values to df
        singleDvDataset["windCapacity"][0] = instance.windCapacity.value
        singleDvDataset["solarCapacity"][0] = instance.solarCapacity.value
            
        singleDvDataset["bsPowerCapacity"][0] = instance.bsPowerCapacity.value
        singleDvDataset["bsEnergyCapacity"][0] = instance.bsEnergyCapacity.value                   
        singleDvDataset["totalSystemCost"][0] = value(instance.SystemCost)
        
        #still assigning single values to df however looking at LCOH and various segments contributing
        totalHydrogenProduction = instance.utilizationRatio*sum((inputDataset["hydrogenDemand"]*365)/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))

        
        singleDvDataset["LCOH"][0] = value(instance.SystemCost)/totalHydrogenProduction
        
        singleDvDataset["windCosts"][0] = (instance.windCapacity.value*(sum((instance.capexWind / (math.pow((1+instance.WACC),j*instance.reLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.reLifetime))))
                                        + sum((instance.fixedOpexWind/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalHydrogenProduction        
        singleDvDataset["windCapexCosts"] = (instance.windCapacity.value*(sum((instance.capexWind / (math.pow((1+instance.WACC),j*instance.reLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.reLifetime))))))/totalHydrogenProduction
        singleDvDataset["windOpexCosts"] = (instance.windCapacity.value*(sum((instance.fixedOpexWind/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalHydrogenProduction
        
        
        
        singleDvDataset["solarCosts"][0] = (instance.solarCapacity.value*(sum((instance.capexSolar / (math.pow((1+instance.WACC),j*instance.reLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.reLifetime))))
                                        + sum((instance.fixedOpexSolar/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalHydrogenProduction        
        singleDvDataset["solarCapexCosts"] = (instance.solarCapacity.value*(sum((instance.capexSolar / (math.pow((1+instance.WACC),j*instance.reLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.reLifetime))))))/totalHydrogenProduction
        singleDvDataset["solarOpexCosts"] = (instance.solarCapacity.value*(sum((instance.fixedOpexSolar/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))))/totalHydrogenProduction
        
        
        
        singleDvDataset["eyCosts"][0] = (sum(instance.stackSize[i]*instance.eyCapacity[i].value*(sum((instance.capexEY[i] / (math.pow((1+instance.WACC),j*instance.eyLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.eyLifetime)))) + 
                    sum((instance.fixedOpexEY[i]/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))))  for i in instance.eyPlants) +
                    sum((8760/len(instance.horizon))*sum(instance.variableOpexEY[i]*instance.eyGen[i,t].value for i in instance.eyPlants)/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime)))/totalHydrogenProduction
        singleDvDataset["eyCapexCosts"] = (sum(instance.stackSize[i]*instance.eyCapacity[i].value*(sum((instance.capexEY[i] / (math.pow((1+instance.WACC),j*instance.eyLifetime)) for j in np.arange(math.ceil(instance.simulationLifetime/instance.eyLifetime)))))  for i in instance.eyPlants))/totalHydrogenProduction
        singleDvDataset["eyOpexCosts"] = (sum(instance.stackSize[i]*instance.eyCapacity[i].value*sum((instance.fixedOpexEY[i]/(math.pow((1+value(instance.WACC)),t))) for t in np.arange(instance.simulationLifetime)) for i in instance.eyPlants) + sum((8760/len(instance.horizon))*sum(instance.variableOpexEY[i]*instance.eyGen[i,t].value for i in instance.eyPlants)/(math.pow((1+value(instance.WACC)),t)) for t in np.arange(instance.simulationLifetime)))/totalHydrogenProduction
        
        
        

        
        singleDvDataset["bsPowerCosts"][0] = (sum(instance.bsPowerCapacity.value*(instance.capexBSpower / (math.pow((1+instance.WACC),j*instance.bsLifetime)))for j in np.arange(math.ceil(instance.simulationLifetime/instance.bsLifetime))) +
                                              instance.bsPowerCapacity.value*sum((instance.fixedOpexBSpower/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))))/totalHydrogenProduction                      
        singleDvDataset["bsEnergyCosts"][0] = (sum(instance.bsEnergyCapacity.value*(instance.capexBSenergy / (math.pow((1+instance.WACC),j*instance.bsLifetime))) for j in np.arange(math.ceil(instance.simulationLifetime/instance.bsLifetime))))/totalHydrogenProduction        
        singleDvDataset["bsPowerCapexCosts"] = (sum(instance.bsPowerCapacity.value*(instance.capexBSpower / (math.pow((1+instance.WACC),j*instance.bsLifetime)))for j in np.arange(math.ceil(instance.simulationLifetime/instance.bsLifetime))))/totalHydrogenProduction
        singleDvDataset["bsPowerOpexCosts"] = (instance.bsPowerCapacity.value*sum((instance.fixedOpexBSpower/(math.pow((1+instance.WACC),t)) for t in np.arange(instance.simulationLifetime))))/totalHydrogenProduction
        
        
        
        
        #assigning hourly values to dfs
        for hour in np.arange(len(inputDataset["cfSolar"])):
            hourlyDvDataset["windGen"][hour] = singleDvDataset["windCapacity"][0]*inputDataset["cfWind"][hour]
            hourlyDvDataset["solarGen"][hour] = singleDvDataset["solarCapacity"][0]*inputDataset["cfSolar"][hour]
            hourlyDvDataset["bsStore"][hour] = instance.bsStore[hour].value    
            hourlyDvDataset["bsAvail"][hour] = instance.bsAvail[hour].value    
            hourlyDvDataset["bsDeploy"][hour] = instance.bsDeploy[hour].value
            
            #for later data analysis
            hourlyDvDataset["timestep"][hour] = hour + 1   

        #assigning single dvs for ey types
        eySingleDvDataset = pd.DataFrame(columns=np.arange(len(inputDataset["capexEY"])), index=range(2))
        for eyUnit in np.arange(len(inputDataset["capexEY"])):
            #looking at capacity and load factor
            eySingleDvDataset[eyUnit][0] = instance.stackSize[eyUnit]*instance.eyCapacity[eyUnit].value
            if(instance.eyCapacity[eyUnit].value != 0):
                eySingleDvDataset[eyUnit][1] = sum(instance.eyGen[eyUnit,hour].value for hour in np.arange(len(instance.horizon)))/(len(instance.horizon)*(1/instance.energyUseEY[eyUnit])*instance.stackSize[eyUnit]*instance.eyCapacity[eyUnit].value)
            else:
                eySingleDvDataset[eyUnit][1] = 0
        #assigning hourly dvs for each ey unit
        for eyUnit in np.arange(len(inputDataset["capexEY"])):
            for hour in np.arange(len(inputDataset["cfSolar"])):
                eyHourlyDvDataset[eyUnit][hour] = instance.eyGen[eyUnit,hour].value          
        
        
        #now saving 4 datasets to different sheets in same excel file
        excelOutputFileName = f"../modelOutputs/{dataFileName}.xlsx"
        with pd.ExcelWriter(excelOutputFileName) as writer:  
            singleDvDataset.to_excel(writer,sheet_name='singleValueDvs') 
            
            hourlyDvDataset.to_excel(writer,sheet_name='hourlyValueDvs') 
            
            eySingleDvDataset.to_excel(writer,sheet_name='singleEyValueDvs') 
            
            eyHourlyDvDataset.to_excel(writer,sheet_name='hourlyEyValueDvs') 
        
        print(f"Model output results saved to {excelOutputFileName}")

In [ ]:
#@title Single Run model {form-width: "15%" }

# # # # START OF SPECIFIC PARAMETER CHANGES # # # # 

#year: 0-2017, 1-2018, 2-2019
year = 0


siteName = "northWest"

# # # # END OF SPECIFIC PARAMETER CHANGES # # # # 


yearList = ["2017","2018","2019"]
testMode = True
startDay = year*365
endDay = (year+1)*365
transmissionLosses = .02

#dont include the .dat ending or path location
datFileName = f"{siteName}_base_{yearList[year]}"

siteLocation = f"{siteName}Site"


pdSingleParamDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='systemSettings')

paramNames = pdSingleParamDataset["ParamName"]
paramValues = pdSingleParamDataset["Value"]

#creating input dataset for single value Params
inputDataset = {}

#adding wind and solar generation capacity factors
windCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=["cfWind"])
solarCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=["cfSolar"])



inputDataset["cfWind"] = np.array(windCFDataset["cfWind"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)

inputDataset["cfSolar"] = np.array(solarCFDataset["cfSolar"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)

#adding single param value data
for paramName,paramValue in zip(paramNames,paramValues):
    inputDataset[paramName] = paramValue


pdEYDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='eyUnitSettings')
#adding EY unit data
for paramName in pdEYDataset.columns:
    if(paramName == "Name"):
        #skip assigning the Name column as the model uses integer indices
        continue
    #create empty array inside dict
    inputDataset[paramName] = np.zeros(len(pdEYDataset["capexEY"]))
    for index in np.arange(0,len(pdEYDataset["capexEY"])):
        inputDataset[paramName][index] = pdEYDataset[paramName][index]

# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Run started:", start_time)

greenHydrogenProduction.main(datFileName,inputDataset,testMode)


#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)

#printing total model runtime
print("Total model time took: ",str(endRun-startRun))

Run started: 16:00:34
Test mode activated but one of files already deleted
Completed data file
Model output results saved to ../modelOutputs/northWest_base_2017.xlsx
Run over: 16:02:16
Total model time took:  0:01:42.795143


In [ ]:
#@title Base Results Model Runs {form-width: "15%" }

#@title Single Run model {form-width: "15%" }

# # # # START OF SPECIFIC PARAMETER CHANGES # # # # 

#year: 0-2017, 1-2018, 2-2019
year = 0


siteName = "northWest"

# # # # END OF SPECIFIC PARAMETER CHANGES # # # # 


yearList = ["2017","2018","2019"]
testMode = True
startDay = year*365
endDay = (year+1)*365
transmissionLosses = .02

#dont include the .dat ending or path location
datFileName = f"{siteName}_base_{yearList[year]}"

siteLocation = f"{siteName}Site"


pdSingleParamDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='systemSettings')

paramNames = pdSingleParamDataset["ParamName"]
paramValues = pdSingleParamDataset["Value"]

#creating input dataset for single value Params
inputDataset = {}

#adding wind and solar generation capacity factors
windCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=["cfWind"])
solarCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=["cfSolar"])



inputDataset["cfWind"] = np.array(windCFDataset["cfWind"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)

inputDataset["cfSolar"] = np.array(solarCFDataset["cfSolar"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)

#adding single param value data
for paramName,paramValue in zip(paramNames,paramValues):
    inputDataset[paramName] = paramValue


pdEYDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='eyUnitSettings')
#adding EY unit data
for paramName in pdEYDataset.columns:
    if(paramName == "Name"):
        #skip assigning the Name column as the model uses integer indices
        continue
    #create empty array inside dict
    inputDataset[paramName] = np.zeros(len(pdEYDataset["capexEY"]))
    for index in np.arange(0,len(pdEYDataset["capexEY"])):
        inputDataset[paramName][index] = pdEYDataset[paramName][index]

# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Run started:", start_time)

greenHydrogenProduction.main(datFileName,inputDataset,testMode)


#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)

#printing total model runtime
print("Total model time took: ",str(endRun-startRun))


In [ ]:
#@title Input Dataset for Batch Runs {form-width: "15%" }

#dont include the .dat ending or path location
datFileName = "northWestSite"
testMode = True
startDay = 1*365
endDay = 2*365
transmissionLosses = .02
siteLocation = "northWestSite"


pdSingleParamDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='systemSettings')

paramNames = pdSingleParamDataset["ParamName"]
paramValues = pdSingleParamDataset["Value"]

#creating input dataset for single value Params
inputDataset = {}

#adding wind and solar generation capacity factors
windCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=["cfWind"])
solarCFDataset = pd.read_excel(f"../dataInputs/sites/{siteLocation}/reData.xlsx",usecols=['cfSolar'])


inputDataset["cfWind"] = np.array(windCFDataset["cfWind"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)

inputDataset["cfSolar"] = np.array(solarCFDataset["cfSolar"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)

#adding single param value data
for paramName,paramValue in zip(paramNames,paramValues):
    inputDataset[paramName] = paramValue


pdEYDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='eyUnitSettings')
#adding EY unit data
for paramName in pdEYDataset.columns:
    if(paramName == "Name"):
        #skip assigning the Name column as the model uses integer indices
        continue
    #create empty array inside dict
    inputDataset[paramName] = np.zeros(len(pdEYDataset["capexEY"]))
    for index in np.arange(0,len(pdEYDataset["capexEY"])):
        inputDataset[paramName][index] = pdEYDataset[paramName][index]



In [ ]:
#@title Single Parameter Batch Run { form-width: "15%" }

def runModel(fileName,inputDataset):
  #running model with specific parameter input
  greenHydrogenProduction.main(fileName,inputDataset,False)


#identifying which parameter we want to run a batch run on
parameterTested = "capexEY"
initialValue = 341000

#range to test
percentRange = [0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5]
#percentRange = [0.8,0.9,1.0]

parameterData =  initialValue*np.array(percentRange)

parameterFileList = []
for percent in percentRange:
  parameterFileList.append(f"{parameterTested}_{percent}")

# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Batch run started:", start_time)

processes = []

# creating processes
for w in range(len(parameterData)):

    #creating unique file structure for name
    fileName = f"{siteLocation}_{parameterFileList[w]}"

    if(parameterTested == "capexEY"):
      #have multiple electroyzers so need to select large one
      inputDataset[parameterTested][1] = parameterData[w]
    else:
      inputDataset[parameterTested] = parameterData[w]

    #changing OPEX costs
    if(parameterTested == "capexSolar"):
      inputDataset["fixedOpexSolar"] = parameterData[w]*.02
    elif(parameterTested == "capexWind"):
      inputDataset["fixedOpexWind"] = parameterData[w]*.02
    elif(parameterTested == "capexBSpower"):
      inputDataset["fixedOpexBSpower"] = inputDataset["fixedOpexBSpower"]*percentRange[w]
    elif(parameterTested == "capexEY"):
      inputDataset["fixedOpexEY"][1] = parameterData[w]*.02

    p = Process(target=runModel, args=(fileName,inputDataset))
    processes.append(p)
    p.start()

# completing process
for p in processes:
    p.join()


####### RESETING VALUES #######

#adding single param value data
for paramName,paramValue in zip(paramNames,paramValues):
    inputDataset[paramName] = paramValue


pdEYDataset = pd.read_excel("../dataInputs/inputSheet.xlsx",sheet_name='eyUnitSettings')
#adding EY unit data
for paramName in pdEYDataset.columns:
    if(paramName == "Name"):
        #skip assigning the Name column as the model uses integer indices
        continue
    #create empty array inside dict
    inputDataset[paramName] = np.zeros(len(pdEYDataset["capexEY"]))
    for index in np.arange(0,len(pdEYDataset["capexEY"])):
        inputDataset[paramName][index] = pdEYDataset[paramName][index]

####### RESETING VALUES #######

#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)

#printing total model runtime
print("Total model time took: ",str(endRun-startRun))



In [ ]:
#@title Different Sites Batch Run { form-width: "15%" }


def runModel(fileName,inputDataset):
  #running model with specific parameter input
  greenHydrogenProduction.main(fileName,inputDataset,False)


#identifying which parameter we want to run a batch run on
siteLocations = ["centralWestSite","northEastSite","northWestSite"]

#setting start day
startDay = 0
endDay = 365
# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Batch run started:", start_time)

processes = []

# creating processes and updating inputdataset each time through
for w in range(len(siteLocations)):
    #adding wind and solar generation capacity factors
    windCFDataset = pd.read_excel(f"../dataInputs/sites/{site}/reData.xlsx",usecols=["cfWind"])
    solarCFDataset = pd.read_excel(f"../dataInputs/sites/{site}/reData.xlsx",usecols=['cfSolar'])

    #changing inputDataset data for correct site
    inputDataset["cfWind"] = np.array(windCFDataset["cfWind"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)
    inputDataset["cfSolar"] = np.array(solarCFDataset["cfSolar"])[(24*startDay):(24*endDay)]*(1-transmissionLosses)

    #getting out correct filename
    fileName = f"{siteLocations[w]}_{startDay}_{endDay}"

    #creating process with updated fileName and inputDataset
    p = Process(target=runModel, args=(fileName,inputDataset))
    processes.append(p)
    p.start()

# completing process
for p in processes:
    p.join()



#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)

#printing total model runtime
print("Total model time took: ",str(endRun-startRun))



In [ ]:
#@title Different Time Periods Batch Run { form-width: "15%" }


def runModel(fileName,inputDataset):
  #running model with specific parameter input
  greenHydrogenProduction.main(fileName,inputDataset,True)


#outlining time periods under study

#outlining time periods under study
timePeriods = np.array([[0,181],[31,212],[59,243],[90,273],[120,304],[151,334],[181,365]])
shrunkenTimePeriods = np.array([[0,181],[90,273],[181,365]])
timePeriods2018 = timePeriods + 365
timePeriods2019 = timePeriods2018 + 365

totalTimePeriods = np.concatenate((timePeriods,timePeriods2018,timePeriods2019))


# for tracking elapsed time
startRun = datetime.now()
start_time = startRun.strftime("%H:%M:%S")
print("Batch run started:", start_time)

processes = []

# creating processes and updating inputdataset each time through
for period in shrunkenTimePeriods:
  #changing inputDataset data for correct site
  inputDataset["cfWind"] = np.array(windCFDataset["cfWind"])[(24*period[0]):(24*period[1])]*(1-transmissionLosses)
  inputDataset["cfSolar"] = np.array(solarCFDataset["cfSolar"])[(24*period[0]):(24*period[1])]*(1-transmissionLosses)

  #getting out correct filename
  fileName = f"northWestSite_{period[0]}_{period[1]}"


  #creating process with updated fileName and inputDataset
  p = Process(target=runModel, args=(fileName,inputDataset))
  processes.append(p)
  p.start()

# completing process
for p in processes:
    p.join()



#getting end time
endRun = datetime.now()
end_time = endRun.strftime("%H:%M:%S")
print("Run over:", end_time)

#printing total model runtime
print("Total model time took: ",str(endRun-startRun))
